In [1]:
%load_ext autoreload
%autoreload 2

import os, json, yaml

import matplotlib.pyplot as plt
import matplotlib.colors as colors

import numpy as np
import scipy as sp
import scipy.stats as stats
import nibabel as nb
import cortex

%matplotlib inline

base_dir = '/Users/knapen/Downloads/prf_lyon/'
os.chdir(base_dir)

with open(os.path.join(base_dir, 'code', 'settings.yml'), 'r') as f:
    analysis_info = yaml.load(f)
    
fs_dir = '/Users/knapen/Downloads/prf_lyon/derivatives/out/freesurfer'

N_PROCS = 8

# trying deperately to optimize for speed
import ctypes
# conda_dir = os.environ['CONDA_PREFIX']
conda_dir = '/Users/knapen/miniconda3/envs/py36'
mkl_rt = ctypes.CDLL(os.path.join(conda_dir, 'lib', 'libmkl_rt.dylib'))
mkl_rt.mkl_set_num_threads(ctypes.byref(ctypes.c_int(N_PROCS)))

Cannot find shapely, using simple label placement


460099216

In [2]:
sub = 'sub-01'

input_file = '/Users/knapen/Downloads/prf_lyon/derivatives/out/pp/sub-01/ses-01/func/sub-01_task-prf_acq-median_T1w_desc-preproc_bold.nii.gz'
dm_file = os.path.join(base_dir, 'code', 'dm_out.npy')
    
mask_file = nb.load('/Users/knapen/Downloads/prf_lyon/derivatives/out/pp/sub-01/ses-01/func/sub-01_task-prf_acq-median_T1w_desc-preproc_brainmask.nii.gz')
mask = mask_file.get_data().astype(bool)

# for registration into pycortex
example_epi_file = '/Users/knapen/Downloads/prf_lyon/derivatives/out/pp/sub-01/ses-01/func/sub-01_task-prf_acq-median_T1w_desc-preproc_boldref.nii.gz'
T1_file = os.path.join(base_dir, 'derivatives/out/fmriprep/', sub, 'anat', \
     sub+'_desc-preproc_T1w.nii.gz')
fs_T1_file = os.path.join('/Users/knapen/Downloads/prf_lyon/derivatives/out/freesurfer/sub-01/mri', \
     'T1.nii.gz')

#design matrix
visual_dm = np.load(dm_file).T

# data
in_file_nii = nb.load(input_file)
data = in_file_nii.get_data().reshape((-1,in_file_nii.shape[-1]))

In [3]:
fit_model = analysis_info["fit_model"]

# Fit: define search grids
x_grid_bound = (-analysis_info["max_eccen"], analysis_info["max_eccen"])
y_grid_bound = (-analysis_info["max_eccen"], analysis_info["max_eccen"])
sigma_grid_bound = (analysis_info["min_size"], analysis_info["max_size"])
n_grid_bound = (analysis_info["min_n"], analysis_info["max_n"])
grid_steps = analysis_info["grid_steps"]

# Fit: define search bounds
x_fit_bound = (-analysis_info["max_eccen"]*2, analysis_info["max_eccen"]*2)
y_fit_bound = (-analysis_info["max_eccen"]*2, analysis_info["max_eccen"]*2)
sigma_fit_bound = (1e-6, 1e2)
n_fit_bound = (1e-6, 2)
beta_fit_bound = (-1e6, 1e6)
baseline_fit_bound = (-1e6, 1e6)

if fit_model == 'gauss' or fit_model == 'gauss_sg':
    bound_grids  = (x_grid_bound, y_grid_bound, sigma_grid_bound)
    bound_fits = (x_fit_bound, y_fit_bound, sigma_fit_bound, beta_fit_bound, baseline_fit_bound)
elif fit_model == 'css' or fit_model == 'css_sg':
    bound_grids  = (x_grid_bound, y_grid_bound, sigma_grid_bound, n_grid_bound)
    bound_fits = (x_fit_bound, y_fit_bound, sigma_fit_bound, n_fit_bound, beta_fit_bound, baseline_fit_bound)

In [4]:
# this is spoofing it, of course. We should create an actual package.
os.chdir(os.path.join(base_dir, 'code'))
from prf_fit import *

In [5]:
grid_steps

20

In [6]:
# intitialize prf analysis
prf = PRF_fit(  data = data[mask.ravel()],
                fit_model = fit_model, 
                visual_design = visual_dm, 
                screen_distance = analysis_info["screen_distance"],
                screen_width = analysis_info["screen_width"],
                scale_factor = 1/4.0, 
                tr =  analysis_info["TR"],
                bound_grids = bound_grids,
                grid_steps = grid_steps,
                bound_fits = bound_fits,
                n_jobs = N_PROCS,
                sg_filter_window_length = analysis_info["sg_filt_window_length"],
                sg_filter_polyorder = analysis_info["sg_filt_polyorder"],
                sg_filter_deriv = analysis_info["sg_filt_deriv"], 
                )
# will need to move/delete this file for new predictions
prediction_file = os.path.join(base_dir, 'derivatives', 'out', 'pp', 'predictions.npy')
if os.path.isfile(prediction_file):
    prf.load_grid_predictions(prediction_file=prediction_file)
else:
    prf.make_predictions(out_file=prediction_file)

plt.figure(figsize=(20,12))
plt.imshow(prf.predictions)

8000it [09:52, 13.50it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/Users/knapen/Downloads/prf_lyon/data/sub-01/predictions.npy'

In [20]:
prf.grid_fit()


100%|██████████| 1000/1000 [5:19:37<00:00, 19.18s/it]


#### Save out outputs

In [21]:
rsq_output = np.zeros(mask_file.shape)
rsq_output[mask] = prf.gridsearch_r2
rsq_out_nii = nb.Nifti1Image(rsq_output, affine=mask_file.affine, header=mask_file.header)
rsq_out_nii.to_filename(os.path.join('/Users/knapen/Downloads/prf_lyon/derivatives/out/pp/sub-01/ses-01/', 'rsq.nii.gz'))

In [24]:
params_output = np.zeros(list(mask_file.shape) + [prf.gridsearch_params.shape[0]])
params_output[mask] = prf.gridsearch_params.T
params_out_nii = nb.Nifti1Image(params_output, affine=mask_file.affine, header=mask_file.header)
params_out_nii.to_filename(os.path.join('/Users/knapen/Downloads/prf_lyon/derivatives/out/pp/sub-01/ses-01/', 'params.nii.gz'))

In [ ]:
prf.gridsearch_params

### Surface stuff

In [25]:
rsq_threshold = 0.1

t = np.sqrt(rsq_threshold)/(np.sqrt((1-rsq_threshold)/(analysis_info['n_timepoints_per_run']-2)))
p = stats.t.sf(t, analysis_info['n_timepoints_per_run']-1)

print('Voxel admission threshold info: \np={p}, t={t}, correlation={r}'.format(p=p, t=t, r=np.sqrt(rsq_threshold)))

Voxel admission threshold info: 
p=4.387246839631265e-06, t=4.570436400267363, correlation=0.31622776601683794


In [35]:
# only run this once for each subject
# cortex.freesurfer.import_subj(
#         subject=sub, sname=sub, freesurfer_subject_dir=fs_dir)

# after having done:
#
# bbregister --s sub-01 --mov fmriprep/sub-01/ses-01/func/sub-01_ses-01_run-1_space-T1w_boldref.nii.gz \
# --reg pp/sub-01/ses-01/register.dat --fslmat pp/sub-01/ses-01/flirt.mtx --init-fsl --bold
#
# and this too, once it works :)
# xfm_data = np.eye(4)
xfm_data = np.loadtxt(os.path.join(base_dir, 'derivatives/out/pp/sub-01/ses-01/flirt.mtx'))
xfm = cortex.xfm.Transform.from_fsl(xfm=xfm_data, 
                                    func_nii=example_epi_file, 
                                    anat_nii=fs_T1_file)
# # Save as pycortex 'coord' transform
xfm.save(sub, 'fmriprep_T1', 'coord')

In [29]:
cortex.database.default_filestore

'/Users/knapen/FS_SJID/cortex/db'

In [37]:
params = nb.load(os.path.join('/Users/knapen/Downloads/prf_lyon/derivatives/out/pp/sub-01/ses-01/', 'params.nii.gz'))
p_data = params.get_data()
rsq = nb.load(os.path.join('/Users/knapen/Downloads/prf_lyon/derivatives/out/pp/sub-01/ses-01/', 'rsq.nii.gz')).get_data()

# now construct polar angle and eccentricity values
complex_location = p_data[...,0] + p_data[...,1] * 1j
polar_angle = np.angle(complex_location)
eccentricity = np.abs(complex_location)

size = p_data[...,2]
# baseline and beta were swapped when running the first fits.
baseline = p_data[...,3]
beta = p_data[...,4]

polar_angle_n = (polar_angle + np.pi) / (np.pi * 2.0)

# make discrete angles for clarity
angle_offset = 0.1
polar_angle_n = np.fmod(polar_angle_n+angle_offset, 1.0)

# convert angles to colors, using correlations as weights
hsv = np.zeros(list(polar_angle_n.shape) + [3])
hsv[..., 0] = polar_angle_n # angs_discrete  # angs_n
hsv[..., 1] = np.ones_like(rsq) # np.sqrt(rsq) #np.ones_like(rsq)  # np.sqrt(rsq)
hsv[..., 2] = np.ones_like(rsq)  # np.sqrt(rsq)# np.ones_like(rsq)

alpha_mask = (rsq <= rsq_threshold).T
alpha = np.sqrt(rsq).T * 5
# alpha[alpha_mask] = 0
alpha = np.ones(alpha.shape)
alpha[alpha_mask] = 0

rgb = colors.hsv_to_rgb(hsv)

In [43]:
vrgba = cortex.VolumeRGB(
    red=rgb[..., 0].T,
    green=rgb[..., 1].T,
    blue=rgb[..., 2].T,
    subject=sub,
    alpha=alpha,
    xfmname='fmriprep_T1')
vecc = cortex.Volume2D(eccentricity.T, rsq.T, sub, 'fmriprep_T1',
                           vmin=0, vmax=10,
                           vmin2=rsq_threshold, vmax2=1.0, cmap='BROYG_2D')
vsize = cortex.Volume2D(size.T, rsq.T, sub, 'fmriprep_T1',
                           vmin=0, vmax=10,
                           vmin2=rsq_threshold, vmax2=1.0, cmap='BROYG_2D')
vbeta = cortex.Volume2D(beta.T, rsq.T, sub, 'fmriprep_T1',
                           vmin=-2.5, vmax=2.5,
                           vmin2=rsq_threshold, vmax2=1.0, cmap='RdBu_r_alpha')
vbaseline = cortex.Volume2D(baseline.T, rsq.T, sub, 'fmriprep_T1',
                           vmin=-1, vmax=1,
                           vmin2=rsq_threshold, vmax2=1.0, cmap='RdBu_r_alpha')
vrsq = cortex.Volume2D(rsq.T, rsq.T, sub, 'fmriprep_T1',
                           vmin=0, vmax=0.8,
                           vmin2=rsq_threshold, vmax2=1.0, cmap='fire_alpha')


ds = cortex.Dataset(polar=vrgba, ecc=vecc, size=vsize, amplitude=vbeta, baseline=vbaseline, rsq=vrsq)
# handle = cortex.webgl.show(data=ds, recache=True, port=12001)
cortex.webgl.make_static(outpath='/Users/knapen/Downloads/prf_lyon/derivatives/out/pp/sub-01/ses-01/webgl', data=ds, recache=True)
# (data=ds, recache=True, port=12001)

ImportError: cannot import name 'stack_context'

In [44]:
ds.save(os.path.join('/Users/knapen/Downloads/prf_lyon/derivatives/out/pp/sub-01/ses-01/', 'pycortex_ds.h5'))
cortex.webgl.make_static(os.path.join('/Users/knapen/Downloads/prf_lyon/derivatives/out/pp/sub-01/ses-01/', 'pycortex_html'), ds)

ImportError: cannot import name 'stack_context'

In [45]:
mask = cortex.get_cortical_mask(sub, 'fmriprep_T1', type='cortical')

0.000%
0.061%
0.122%
0.184%
0.245%
0.306%
0.367%
0.428%
0.490%
0.551%
0.612%
0.673%
0.735%
0.796%
0.857%
0.918%
0.979%
1.041%
1.102%
1.163%
1.224%
1.285%
1.347%
1.408%
1.469%
1.530%
1.591%
1.653%
1.714%
1.775%
1.836%
1.898%
1.959%
2.020%
2.081%
2.142%
2.204%
2.265%
2.326%
2.387%
2.448%
2.510%
2.571%
2.632%
2.693%
2.755%
2.816%
2.877%
2.938%
2.999%
3.061%
3.122%
3.183%
3.244%
3.305%
3.367%
3.428%
3.489%
3.550%
3.611%
3.673%
3.734%
3.795%
3.856%
3.918%
3.979%
4.040%
4.101%
4.162%
4.224%
4.285%
4.346%
4.407%
4.468%
4.530%
4.591%
4.652%
4.713%
4.774%
4.836%
4.897%
4.958%
5.019%
5.081%
5.142%
5.203%
5.264%
5.325%
5.387%
5.448%
5.509%
5.570%
5.631%
5.693%
5.754%
5.815%
5.876%
5.937%
5.999%
6.060%
6.121%
6.182%
6.244%
6.305%
6.366%
6.427%
6.488%
6.550%
6.611%
6.672%
6.733%
6.794%
6.856%
6.917%
6.978%
7.039%
7.100%
7.162%
7.223%
7.284%
7.345%
7.407%
7.468%
7.529%
7.590%
7.651%
7.713%
7.774%
7.835%
7.896%
7.957%
8.019%
8.080%
8.141%
8.202%
8.264%
8.325%
8.386%
8.447%
8.508%
8.570%
8.631%
8.692%

In [46]:
epi_nii = nb.load(example_epi_file)
dims = epi_nii.shape

mask_img = nb.Nifti1Image(dataobj=mask.transpose((2,1,0)), affine=epi_nii.affine, header=epi_nii.header)

cortical_mask_file = os.path.join('/Users/knapen/Downloads/prf_lyon/derivatives/out/pp/sub-01/ses-01/', \
     sub+'_task-prf_dir-AP_cortical_mask.nii.gz')
mask_img.to_filename(cortical_mask_file)

In [14]:
md = in_file_nii.get_data()
md[~mask.transpose((2,1,0)),:] = 0

masked_img = nb.Nifti1Image(dataobj=md, affine=epi_nii.affine, header=epi_nii.header)
masked_img.to_filename(os.path.join(base_dir, 'data', sub, 'func', \
    sub+'_task-prf_acq-median_masked_T1w_desc-preproc_bold.nii.gz'))